In [ ]:
!pip install plotly;

In [ ]:
!pip install --upgrade notebook plotly


In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv('world_population.csv')
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
# Group by columns
population = df.groupby(['Continent', 'Country/Territory', 'CCA3'])[['2022 Population',
                                                                   '2020 Population',
                                                                   '2015 Population',
                                                                   '2010 Population',
                                                                   '2000 Population',
                                                                   '1990 Population',
                                                                   '1980 Population',
                                                                   '1970 Population']
                                                                 ].count().reset_index()


In [ ]:
#Transfom the columns into rows
pop_growth = population.melt(id_vars = ['Continent','Country/Territory','CCA3'],
                             value_vars = ['2022 Population',
                                           '2020 Population',
                                           '2015 Population',
                                           '2010 Population',
                                           '2000 Population',
                                           '1990 Population',
                                           '1980 Population',
                                           '1970 Population'],
                            var_name = 'Years')

In [ ]:
#rename column 'value' to 'Population'
pop_growth.rename(columns = {'value':'Population'}, inplace = True)

#split the year from Growth column
pop_growth['Year'] = pop_growth['Years'].str.split(' ').str[0]

#sort Year column
pop_gr = pop_growth.sort_values(by ='Year', ascending = True)

In [ ]:
fig = px.choropleth(pop_gr, 
                    locations="CCA3", 
                    color="Population", 
                    hover_name = 'Country/Territory',
                    animation_frame="Year", 
                    animation_group="CCA3")
fig.show(renderer="notebook")

In [ ]:
con_pop = pop_gr.groupby(['Continent','Year']).Population.sum().reset_index()

In [ ]:
fig = px.bar(con_pop, x="Continent", 
             y="Population", 
             color="Continent",
             animation_frame="Year", 
             animation_group="Continent", 
             range_y=[0,5000000000])
fig.show()

In [ ]:
def Continent_population(value):
    country_pop = pop_gr[pop_gr.Continent == value]
    
    #set the custom range for each Continent to avoid difficult to see country when popular too low
    if value == 'Asia': number = 1500000000
    if value == 'Europe': number = 150000000
    if value == 'Africa': number = 230000000
    if value == 'Oceania': number = 27000000
    if value == 'North America': number = 350000000
    if value == 'South America': number = 240000000
        
    #custom title
    title = 'Country population in '+value+' continent'
    
    return px.bar(country_pop, 
             x="Country/Territory", 
             y="Population", 
             color = 'Country/Territory',
             animation_frame="Year", 
             animation_group="Country/Territory",
             title = title,
             range_y=[0,number])

#Call the fucntion above to display the growth in Asia's Continent
print(f'Here are the Continent you can pass in to function to diplay the growth: \n{df.Continent.unique()}')

Continent_population('Asia')
#Continent_population('Europe')
#Continent_population('Africa')

In [ ]:
def pop_by_year(value):
    pop_year = pop_gr[pop_gr.Year == value]
    title = 'Population each country in '+value
    return px.sunburst(pop_year,
                       path=['Continent', 'Country/Territory'], 
                       values='Population',
                       color ='Population',
                       title = title)
pop_by_year('2022')

In [ ]:
fig = px.choropleth(df, 
                    locations='CCA3', 
                    color='Area (km²)', 
                    hover_name='Country/Territory', 
                    title='Country area')
fig.show()

In [ ]:
print('Top 10 country which have highest growth rate')

fig = px.line(df,
             x='Country/Territory',
             y='Growth Rate',
             labels = {
                 'Country/Territory' : 'Country'
             },
             title = 'Top 10 country which have highest growth rate',
             color='Continent',
             markers=True)
fig.show()